Original Source: https://huggingface.co/blog/llama2

In [11]:
#!pip install --upgrade peft transformers bitsandbytes accelerate torch

In [12]:
import torch
from accelerate import Accelerator
from datasets import load_dataset
from peft import LoraConfig
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import SFTTrainer

In [13]:
dataset_name = "timdettmers/openassistant-guanaco"
dataset_text_field = "text"
seq_length = 512
# model_path = "/Users/bsantanna/dev/workspace/community/Llama-2-7b-chat-hf"
model_path = "meta-llama/Llama-2-7b-chat-hf"

In [14]:
compute_dtype = getattr(torch, "float16")
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)

training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=1.41e-5,
    logging_steps=1,
    num_train_epochs=3,
    max_steps=-1,
    report_to=["wandb"],
    save_steps=100,
    save_total_limit=10,
    push_to_hub=False,
    hub_model_id=None,
)

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    inference_mode=False
)

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)
tokenizer.add_special_tokens({"pad_token": "<pad>"})
tokenizer.padding_side = "left"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=quantization_config,
    device_map={"": Accelerator().local_process_index},
    torch_dtype=torch.bfloat16
)
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False

dataset = load_dataset(dataset_name, split="train")

ImportError: Using `load_in_8bit=True` requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes `pip install -i https://test.pypi.org/simple/ bitsandbytes` or pip install bitsandbytes` 

In [16]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    max_seq_length=seq_length,
    train_dataset=dataset,
    dataset_text_field=dataset_text_field,
    peft_config=peft_config,
)

NameError: name 'model' is not defined

In [ ]:
trainer.train()